# Non-linear Registration
As we saw in the last section, affine transformations alone are not enough to accurately register individual subjects to a common template space. In order to move beyond simple affine approaches, we have to consider methods where we can move each voxel *independently*. These are known as *non-linear* registration techniques. Generally, these methods are more complex than affine methods and so we will keep the detail presented here limited. However, much of the image registration theory remains relevant. We still have a *reference* image (the MNI templates) that we want to align our *source* image to. In order to do so, we still have to estimate some form of *mapping* between the source and the reference images, and this will require optimisation using some form of *objective function*. We then need to *resample* the source image to move it into standard space. The main difference is that our mapping is no longer given by a 12-parameter affine transformation. Instead, our mapping is based on a *deformation field*.​​​

## Deformation Fields
At its most basic, a deformation field is an image where each voxel value indicates how to move that specific voxel when warping to standard space. There are therefore three of these images, one for the $x$-axis, $y$-axis and $z$-axis. In `SPM`, the deformation field images contain the new millimetre coordinates for each voxel after warping. So the $X$ deformation field contains all the new $x$-coordinates, the $Y$ deformation field contains all the new $y$-coordinates and the $Z$ deformation field contains all the new $z$-coordinates. So rather than using the header matrices to calculate the millimetre coordinates, we can do it directly by looking at the deformation fields. We can then convert these into voxel coordinates in the source image to give us the values we need to copy into our new normalised image. Of course, given how flexible the non-linear approach is, this will almost always need some form of interpolation. 

An examples of a deformation field for the $x$-axis is shown below. As you navigate around the image, compare the $x$-coordinate in red to the voxel value in white. The $x$-coordinate gives the original world-space location of that voxel and the voxel value in white gives the *new* world-space coordinate where the voxel must be moved in order to warp the image to standard space.

<iframe src="deform-x.html" width="800" height="600" frameborder="0" scrolling="no" title=""></iframe>